# ASCII Art from an Image (Bonus: DynamoDB)

## DynamoDB: Amazon's AWS NoSQL database
Learn more about installing DynamoDB locally here: [Running DynamoDB on Your Computer](http://docs.aws.amazon.com/amazondynamodb/latest/developerguide/DynamoDBLocal.html)

**To launch the local database:** ```java -Djava.library.path=./DynamoDBLocal_lib -jar DynamoDBLocal.jar -sharedDb```

### We'll use PynamoDB package, which is an ORM-like for DynamoDB

In [ ]:
from proj.models import AsciiTask
AsciiTask.delete_table()
AsciiTask.create_table(read_capacity_units=1, write_capacity_units=1)

Open the DynamoDB shell: [Link](http://localhost:8000/shell)

```Execute``` the following script:

<pre>var params = {
    TableName: "dynamodb-ascii-tasks"
};

docClient.scan(params, function(err, data) {
    if (err)
        console.log(JSON.stringify(err, null, 2));
    else
        console.log(JSON.stringify(data, null, 2));
        console.log('Count of records: ' + JSON.stringify(data.Count, null, 2));
});</pre>

---

## A couple helper functions
Note that there is an **improved** ```image_to_ascii_task``` wrapper

In [ ]:
from proj.tasks.ascii import image_to_ascii_task_improved  # Note: this is slightly improved
from proj.helpers import print_ascii_html, display_progress_bar_until_completed

In [ ]:
def queue_and_return_ascii_task(filename, columns):
    task = image_to_ascii_task_improved.apply_async(args=[filename, columns])
    return image_to_ascii_task_improved.AsyncResult(task.id)    

## Queue all the things!

In [ ]:
images = [
    dict(image_uri='samples/images/cat01.jpg', columns=400),
    dict(image_uri='samples/images/cat02.jpg', columns=400),
    dict(image_uri='samples/images/cat03.jpg', columns=400),
    dict(image_uri='samples/images/cat04.jpg', columns=400),
    dict(image_uri='samples/images/cat05.jpg', columns=400),
    dict(image_uri='samples/images/cat06.jpg', columns=400),
    dict(image_uri='samples/images/cat07.jpg', columns=400),
    dict(image_uri='samples/images/doom_xboxone_cover.jpg', columns=200),
]

tasks = []

for image in images:
    tasks.append(queue_and_return_ascii_task(image.get('image_uri'), image.get('columns')))

## Wait until all items are done

In [ ]:
for task in tasks:
    print(task)
    display_progress_bar_until_completed(task)
print("ALL DONE!")

## Load a random item from the database

In [ ]:
from random import choice
ascii_tasks = list(AsciiTask.scan())
task_choice = choice(ascii_tasks)
print_ascii_html(task_choice.ascii_text, font_size_pct=15, line_height_pct=100)

## Query DynamoDB to see there are three records

In [ ]:
AsciiTask.count()

## Load the DynamoDB shell again:

Open the DynamoDB shell: [Link](http://localhost:8000/shell)

```Execute``` the following script:

<pre>var params = {
    TableName: "dynamodb-ascii-tasks"
};

docClient.scan(params, function(err, data) {
    if (err)
        console.log(JSON.stringify(err, null, 2));
    else
        console.log(JSON.stringify(data, null, 2));
        console.log('Count of records: ' + JSON.stringify(data.Count, null, 2));
});</pre>


## Notice that there are now 3 items stored in the database

---

## Now print all items in the database

In [ ]:
for ascii_task in AsciiTask.scan():
    print(ascii_task.task_id)
    print_ascii_html(ascii_task.ascii_text, font_size_pct=15, line_height_pct=100)